# LightGBM

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain-tools/blob/main/examples/langchain-tools_LightGBM.ipynb)

In [4]:
!pip install git+https://github.com/fuyu-quant/langchain-tools.git

  Cloning https://github.com/fuyu-quant/langchain-tools.git to /tmp/pip-req-build-w91zpwuw
  Running command git clone --filter=blob:none --quiet https://github.com/fuyu-quant/langchain-tools.git /tmp/pip-req-build-w91zpwuw
  Resolved https://github.com/fuyu-quant/langchain-tools.git to commit 045842614ed1f586e5bca508fd3448993626823b
  Preparing metadata (setup.py) ... done


In [2]:
%%capture
!pip install langchain==0.0.152
!pip install openai==00.27.5
!pip install lightgbm==3.3.5

In [3]:
import pandas as pd
import lightgbm as lgbm
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# LangChain
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool


# langchaintools 
from langchaintools import mltools


import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

### データセットの用意
* 現在の実装では学習や推論を行うのにcsvファイルにする必要があります

In [ ]:
iris = load_iris()
x = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
print(f'全データ数:{len(x)}')

train, test, train_target, test_target = train_test_split(x, y ,test_size = 0.2, random_state=3655)

train['target'] = train_target
#test['target'] = test_target

print(f'学習データ数:{len(train)}')
print(f'学習データ数:{len(test)}')
train.to_csv('/content/train.csv')
test.to_csv('/content/test.csv')

### langchin-toolsの設定

In [ ]:
# Toolの設定
tools = [mltools.LgbmtrainTool(), mltools.LgbminferenceTool()]

# 通常のLangChainの設定
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

### LightGBMの学習と推論

In [ ]:
agent.run("train.csvを使ってLightGBMの学習を行なったあとtest.csvのデータを推論してください")

### 精度の確認

In [ ]:
# 推論結果の読み込み
test_pred = pd.read_csv('/content/inference.csv', index_col = 0)
test_pred = test_pred.values.argmax(axis = 1)

score = accuracy_score(test_target, test_pred)
print(f'予測精度:{score*100}%')